### Mounting Drive and Changing Directory

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

# Change to your folder in Drive
os.chdir('/content/drive/My Drive/Chess_AI/Model_Training')
print(os.getcwd())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Chess_AI/Model_Training


In [ ]:
ls

ChessBot_SupervisedLearning.ipynb  DataSet/  stockfish/


### Loading Data

In [1]:
import os
path = r"DataSet/"
files = [file for file in os.listdir(path)]

In [2]:
print("Files are: ",len(files))

Files are:  79


### Downloading Required Libraries

In [8]:
!pip install chess
!pip install tqdm
!pip install tensorflow
!pip install numpy


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### Using Chess Lib to Import PGN

In [3]:
from chess import pgn, Board


def load_pgn(file_path):
    games = []
    with open(file_path, "r") as pgn_file:
        while True:
            game = pgn.read_game(pgn_file)
            if game is None:
                break
            else:
                games.append(game)
    return games


In [ ]:
from tqdm import tqdm

# Memory Limit:
limit_files = 16
games = []
i = 1
for file in tqdm(files):
    games.extend(load_pgn(os.path.join(path, file)))
    if i >= limit_files:
        break
    i+=1

 19%|█▉        | 15/79 [00:11<00:48,  1.33it/s]


In [227]:
print(len(games))
print(type(games[0]))
print() # Extra Line
print(games[0])

3356
<class 'chess.pgn.Game'>

[Event "Rated Blitz game"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "Matetricks"]
[Black "nicky"]
[Result "1-0"]
[WhiteTitle "NM"]
[WhiteElo "2471"]
[BlackElo "2211"]
[ECO "B01"]
[Opening "Scandinavian Defense: Main Line"]
[TimeControl "180+0"]
[UTCDate "2013.09.25"]
[UTCTime "00:38:04"]
[Termination "Normal"]
[WhiteRatingDiff "+7"]
[BlackRatingDiff "-6"]

1. e4 d5 2. exd5 Qxd5 3. Nc3 Qa5 4. d4 c6 5. Nf3 Nf6 6. Bc4 Bg4 7. h3 Bh5 8. O-O e6 9. g4 Nxg4 10. hxg4 Bxg4 11. Be2 Bd6 12. Ne4 Bc7 13. Nc5 Bd6 14. Nxb7 Qc7 15. Nxd6+ Qxd6 16. Qd2 Bf5 17. c4 h5 18. Qf4 Qb4 19. d5 cxd5 20. cxd5 Qxf4 21. Bxf4 exd5 22. Nd4 Bd7 23. Bf3 Na6 24. Rfe1+ Kf8 25. Bxd5 Re8 26. Bd6+ Kg8 27. Rxe8+ Bxe8 28. Re1 Kh7 29. Re7 1-0


## Training Nural Network

In [9]:
import numpy as np
from chess import Board

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
print("Imports OK")


Imports OK


In [223]:
# Utility Functions
def board_to_matrix(board: Board):
    matrix = np.zeros((8,8,12))
    piece_map = board.piece_map()

    for square, piece in piece_map.items():
        row,col = divmod(square,8)

        # for channel selection

        piece_type = piece.piece_type - 1 # using zero-based channels
        piece_color = 0 if piece.color else 1

        channel = piece_type + piece_color

        matrix[row][col][channel] = 1
    return matrix

def create_input_for_nn(games):
    X = [] # Collecting all moves for the game
    y = [] # will be using this to collect actual answers
    for game in games:
        board = game.board()
        for move in game.mainline_moves():
            X.append(board_to_matrix(board))
            y.append(move.uci())
            board.push(move)

    return X,y

def encode_moves(moves):
    # that is making uci-format as Dict--> {e2e4:1} unique number for each
    move_to_int = {move:idx for idx, move in enumerate(set(moves))}
    # set to avoid duplications

    return [move_to_int[move] for move in moves], move_to_int

    # here returning::
    # [0,1,2,3,4,5,6,---] and {"move-uci" : <unique-index>}
    # like --> {"e2e4" : 3, ---, so-on}

In [228]:
# Input and Output-Validation for NN
X,y = create_input_for_nn(games)
y,move_to_int = encode_moves(y)
y = to_categorical(y, num_classes=len(move_to_int))
X = np.array(X)


MemoryError: Unable to allocate 3.88 GiB for an array with shape (283583, 1835) and data type float64

In [ ]:
## Nural Network Structure and DataFitting:
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(8, 8, 12)),
    Conv2D(128, (3, 3), activation='relu'),
    Flatten(), # here it's being used to turn 3D output to 1D for Dense Layer
    Dense(256, activation='relu'),
    Dense(len(move_to_int), activation='softmax')
])
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
# Tracks Accuracy Matrics each Epoch and backtracing to increasing it
epochs = 50
model.summary()
history = model.fit(X, y, epochs=epochs, validation_split=0.1, batch_size=64)
model.save(f"models/TF_{epochs}EPOCHS_{limit_files}.keras")

# for Importing into another file (it's needed, see prediction area to Understand)
import json
with open(f"move_to_ints_files/move_to_int_{epochs}_{limit_files}.json", "w") as f:
    json.dump(move_to_int, f)

# To Load:
# with open("move_to_int.json", "r") as f:
#     move_to_int = json.load(f)

c:\Users\Romey Si\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 6, 6, 64)       │         6,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1841)           │       473,137 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,078,513 (4.11 MB)

 Trainable params: 1,078,513 (4.11 MB)

 Non-trainable params: 0 (0.00 B)

MemoryError: Unable to allocate 2.15 GiB for an array with shape (314189, 1841) and data type float32

### Predictions

In [19]:
int_to_move = {v:k for k,v in move_to_int.items()}

def predict_next_move(board):

    board_matrix = board_to_matrix(board).reshape(1, 8, 8, 12)
    predictions = model.predict(board_matrix)[0]

    legal_moves = list(board.legal_moves)
    legal_moves_uci = [move.uci() for move in legal_moves]

    sorted_indices = np.argsort(predictions)[::-1]

    for move_index in sorted_indices:
        move = int_to_move[move_index]
        if move in legal_moves_uci:
            return move
    return None

## Notes:
# 1 Convert the board to input format expected by the model
# 2 Get model predictions (a probability for each move class)
# 3 Get all legal moves in the current position
# 4 Sort move indices from most to least probable
# 5 Go through top predicted moves and return the first legal one


### Gameplay

In [20]:
board = Board() # Starting Board for Game

In [218]:
print("Board before prediction:")
print(board)

# Predict and make the move
next_move = predict_next_move(board)
board.push_uci(next_move)

# Display the board after prediction
print("\nPredicted move:", next_move)
print("Board after prediction:")
print(board)

Board before prediction:
. . . . n . k .
. p . P . . r .
. P . . . p . .
p . . . . K . .
. . . . . b . .
. . . . . . . .
. . . . . . . p
. . . . . . . .
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step

Predicted move: f5e6
Board after prediction:
. . . . n . k .
. p . P . . r .
. P . . K p . .
p . . . . . . .
. . . . . b . .
. . . . . . . .
. . . . . . . p
. . . . . . . .


In [219]:
# PGN Showcase
print(str(pgn.Game.from_board(board)))

[Event "?"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "?"]
[Black "?"]
[Result "1/2-1/2"]

1. e4 c5 2. Nf3 d6 3. d4 cxd4 4. Nxd4 Nf6 5. Nc3 g6 6. Be3 Bg7 7. f3 O-O 8. Qd2 Nc6 9. Bc4 Bd7 10. O-O-O Rc8 11. h4 Ne5 12. h5 Nxc4 13. g4 Qa5 14. hxg6 fxg6 15. Kb1 Rc7 16. g5 d5 17. e5 Ne8 18. Rhe1 h5 19. e6 Nb6 20. Qd3 Bxd4 21. Bd2 Ba4 22. Nb5 Nd7 23. Bb4 Nc5 24. Qf5 Nxe6 25. Re5 Bxe5 26. a3 Qxb4 27. f4 Qc5 28. c3 Qd6 29. Qf6 Qd7 30. f5 Rc6 31. fxg6 Bxb5 32. c4 a6 33. cxd5 Rc5 34. Rd2 Rxf6 35. b3 Rc8 36. Ka2 h4 37. Rd3 Qd6 38. Rd4 Rc3 39. b4 Rcf3 40. g7 Re3 41. Kb2 Nxd4 42. gxf6 Nb3+ 43. Kb1 exf6 44. a4 h3 45. a5 Re4 46. Kc2 Qe7 47. Kxb3 Bh2 48. d6 Qd8 49. d7 Re5 50. Kc3 Ba4 51. Kc4 Re1 52. b5 Kxg7 53. b6 Bg3 54. Kb4 Bf4 55. Kxa4 Re7 56. Ka3 h2 57. Ka4 Kg8 58. Kb3 Rg7 59. Kb4 Kh7 60. Kc5 Kg8 61. Kc4 Qxb6 62. axb6 Kh7 63. Kc5 Kg8 64. Kd5 Kh7 65. Ke6 a5 66. Kf5 Kg8 67. Ke6 Kh7 68. Kf5 Kg8 69. Ke6 Kh7 70. Kf5 Kg8 71. Ke6 Kh7 72. Kf5 Kg8 73. Ke6 Kh7 74. Kf5 Kg8 75. Ke6 Kh7 76. Kf5 Kg8 77. K